# User Authentication using OAuth2 and APIM fronting Azure OpenAI

In [5]:
tenant_id = "f69a7636-4db8-498c-8ff6-bc7f1aafcec0" # my MCAPS tenant
client_id = "c2933b1e-ba50-4177-ac78-11a564edf49d" # web-lliybm23riihe-docker client id
app_endpoint = "https://web-lliybm23riihe-docker.azurewebsites.net"


In [ ]:
%pip install msal
%pip install openai
%pip install requests
%pip install logging
%pip install json

##  Create a device flow to get the access token

In [ ]:
# Step 1: Install MSAL
# pip install msal

import msal

# Step 2: Configure MSAL
authority = 'https://login.microsoftonline.com/f69a7636-4db8-498c-8ff6-bc7f1aafcec0'
# below is the managed identity for APIM
scopes = ["c2933b1e-ba50-4177-ac78-11a564edf49d"] # , "User.Read"]

# Create a public client application
app = msal.PublicClientApplication(client_id, authority=authority)

# Step 3: Initiate Device Flow
flow = app.initiate_device_flow(scopes=scopes)
if 'user_code' not in flow:
    raise ValueError("Failed to create device flow. Check your client ID and authority.")

print(flow['message'])

In [ ]:
# Step 4: Acquire Token
result = app.acquire_token_by_device_flow(flow)
if 'access_token' in result:
    print("Access token acquired.")
else:
    raise ValueError("Failed to acquire token. Error: {}".format(result.get('error_description')))

access_token = result['access_token']

# Step 5: Validate Token (Optional)
# You can decode the token and check the audience if needed
import jwt

decoded_token = jwt.decode(access_token, options={"verify_signature": False})
audience = decoded_token.get('aud')
# if audience != 'api://your-apim-instance':
#     raise ValueError("Token audience does not match the expected audience.")

print(decoded_token)
print(audience )

In [ ]:
print(audience)
print(access_token)

## Test the API using a direct HTTP call

In [ ]:
import requests
import json

url = app_endpoint + "/api/conversation"

messages={"messages":[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}
response = requests.post(url, headers = { 'Authorization': 'Bearer ' + access_token}, json = messages)
print("status code: ", response.status_code)
if (response.status_code == 200):
    data = json.loads(response.text)
    print("response: ", data.get("choices")[0].get("message").get("content"))
else:
    print(response.text)

## Test the API using the Azure OpenAI Python SDK

In [ ]:
from openai import AzureOpenAI

messages=[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]
client = AzureOpenAI(
    azure_endpoint=apim_endpoint,
    api_key=apim_key,
    api_version=openai_api_version        
)
response = client.chat.completions.create(model=openai_deployment_name, messages=messages, extra_headers={"Authorization": "Bearer " + access_token})
print(response.choices[0].message.content)